# Setup

The first step is to clone the fork, and checkout the branch with the 3LC integration. Set your preferred path, and the next cells will get it cloned and set up.

In [2]:
YOLOV5_PATH = "../../tlc-yolov5"
TLC_PATH = "../tlc-monorepo"

In [3]:
%cd "{YOLOV5_PATH}"

c:\Users\47468\repos\tlc-yolov5


In [ ]:
!git clone "https://3lc-ai@dev.azure.com/3lc-ai/TLC/_git/tlc-yolov5"
!git checkout "feature/frederik.mellbye/tlc-integration"

Now that we have the YOLOv5 repository with the 3LC integration cloned, we need to install the requirements.

In [ ]:
%pip install --upgrade pip

In [ ]:
%pip install -r requirements.txt

If you are on Windows and have a GPU it is a good idea to get the correct PyTorch.

In [ ]:
%pip install torchvision --index-url https://download.pytorch.org/whl/cu118 --force-reinstall

In [ ]:
%pip install -e "{TLC_PATH}"
%pip install poetry

# Training

With everything set up we are now ready to run training with 3LC metrics collection. Let's first take a look at the extra 3LC command line arguments.

In [4]:
!python train.py --help

usage: train.py [-h] [--weights WEIGHTS] [--cfg CFG] [--data DATA] [--hyp HYP]
                [--epochs EPOCHS] [--batch-size BATCH_SIZE] [--imgsz IMGSZ]
                [--rect] [--resume [RESUME]] [--nosave] [--noval]
                [--noautoanchor] [--noplots] [--evolve [EVOLVE]]
                [--bucket BUCKET] [--cache [CACHE]] [--image-weights]
                [--device DEVICE] [--multi-scale] [--single-cls]
                [--optimizer {SGD,Adam,AdamW}] [--sync-bn] [--workers WORKERS]
                [--project PROJECT] [--name NAME] [--exist-ok] [--quad]
                [--cos-lr] [--label-smoothing LABEL_SMOOTHING]
                [--patience PATIENCE] [--freeze FREEZE [FREEZE ...]]
                [--save-period SAVE_PERIOD] [--seed SEED]
                [--local_rank LOCAL_RANK] [--tlc-disable-mc]
                [--tlc-mc-interval TLC_MC_INTERVAL]
                [--tlc-mc-start TLC_MC_START] [--tlc-mc-before-training]
                [--tlc-mc-iou-thres TLC_MC_IOU_THRES

Lets train a YOLOv5n model on COCO128 for ten epochs, collecting metrics every five epochs starting after the fifth. 3LC will automatically read the dataset, and present it back to YOLOv5 which performs training and calls metrics collection. This will result in two 3LC Tables, for the train and validation splits, along with a Run associated with the Tables which contains the collected metrics.

In [ ]:
!python train.py --img 640 --batch 32 --epochs 10 --data coco128.yaml --weights yolov5n.pt --nosave --tlc-mc-start 4 --tlc-mc-interval 5 --device 0

It is also possible to collect metrics independently of training with `collect.py`.

In [5]:
!python collect.py --help

usage: collect.py [-h] [--data DATA] [--weights WEIGHTS [WEIGHTS ...]]
                  [--imgsz IMGSZ] [--conf-thres CONF_THRES]
                  [--iou-thres IOU_THRES] [--max-det MAX_DET] [--split SPLIT]
                  [--device DEVICE] [--workers WORKERS] [--single-cls]
                  [--augment] [--half] [--dnn] [--tlc-iou-thres TLC_IOU_THRES]
                  [--tlc-revision-url TLC_REVISION_URL]

optional arguments:
  -h, --help            show this help message and exit
  --data DATA           dataset.yaml path
  --weights WEIGHTS [WEIGHTS ...]
                        model path(s)
  --imgsz IMGSZ, --img IMGSZ, --img-size IMGSZ
                        inference size (pixels)
  --conf-thres CONF_THRES
                        confidence threshold
  --iou-thres IOU_THRES
                        NMS IoU threshold
  --max-det MAX_DET     maximum detections per image
  --split SPLIT         Split to collect metrics for
  --device DEVICE       cuda device, i.e. 0 or 0,1,2,3 o

In [ ]:
!python collect.py --data coco128.yaml --weights yolov5n.pt --img 640 --device 0

Now you should see both your runs in the 3LC Dashboard, look for the names you see in the outputs above.